## Imports

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
from copy import copy, deepcopy
from typing import List, Sequence

## Problem 1

### Question 1 - Draw a Decision Tree based on Boolean functions

In [2]:
from graphviz import Digraph

class Node:
    def __init__(self, label):
        self.label = label

YES = Node("YES")
NO = Node("YES")

1. A ∧ B ̄ ∧ C

In [3]:
graph = Digraph(format='png', graph_attr={'rankdir': 'TB'})
A, B, C = Node('A'), Node('B'), Node('C')
#no_1, no_2, no_3 = Node('No'), Node('No'), Node('No')
nodes = [A, B, C]
#for n in nodes:
#    graph.node(name=str(id(n)), label=n.label, shape='record')
edges = (
    (A, 'True', B),
    (A, 'False', Node('NO')),
    (B, 'True', Node('NO')),
    (B, 'False', C),
    (C, 'True', Node('YES')),
    (C, 'False', Node('NO')),
    )

for edge_info in edges:
    n1, n2 = edge_info[0], edge_info[-1]
    graph.node(name=str(id(n1)), label=n1.label, shape='record')
    graph.node(name=str(id(n2)), label=n2.label, shape='record')
    e_label = edge_info[1]
    graph.edge(str(id(n1)), str(id(n2)), label=str(e_label))
graph.render('q1_1')

'q1_1.png'

2. X ∧ Y ̄ ∨ X ̄ ∧ Y

In [4]:
graph = Digraph(format='png', graph_attr={'rankdir': 'TB'})
X, Y1, Y2 = Node('X'), Node('Y'), Node('Y')
nodes = [X, Y1, Y2]
edges = (
    (X, 'True', Y1),
    (Y1, 'True', Node('NO')),
    (Y1, 'False', Node('YES')),
    (X, 'False', Y2),
    (Y2, 'True', Node('YES')),
    (Y2, 'False', Node('NO')),
    )

for edge_info in edges:
    n1, n2 = edge_info[0], edge_info[-1]
    graph.node(name=str(id(n1)), label=n1.label, shape='record')
    graph.node(name=str(id(n2)), label=n2.label, shape='record')
    label = edge_info[1]
    graph.edge(str(id(n1)), str(id(n2)), label=label)
graph.render('q1_2')

'q1_2.png'

3. X ∧ Y ∧ Z ∨ X ∧ Y ̄ ∧ W ∨ X ̄ ∧ Y

In [5]:
graph = Digraph(format='png', graph_attr={'rankdir': 'TB'})
X, Y1, Y2, Z, W = Node('X'), Node('Y'), Node('Y'), Node('Z'), Node('W')
nodes = [X, Y1, Y2]
edges = (
    (X, 'True', Y1),
    (Y1, 'True', Z),
    (Z, 'True', Node('YES')),
    (Z, 'False', Node('NO')),
    (Y1, 'False', W),
    (W, 'True', Node('YES')),
    (W, 'False', Node('NO')),
    (X, 'False', Y2),
    (Y2, 'True', Node('YES')),
    (Y2, 'False', Node('NO')),
    )

for edge_info in edges:
    n1, n2 = edge_info[0], edge_info[-1]
    graph.node(name=str(id(n1)), label=n1.label, shape='record')
    graph.node(name=str(id(n2)), label=n2.label, shape='record')
    label = edge_info[1]
    graph.edge(str(id(n1)), str(id(n2)), label=label)
graph.render('q1_3')

'q1_3.png'

### Question 2 - Root node selection using Information Gain and Gini

#### Preliminary steps

##### We first conver the pdf table to one we can use in programming to make our life easier

In [6]:
from pypdf import PdfReader

In [7]:
reader = PdfReader("IDAI610_PS1_DecisionTree.pdf")
page = reader.pages[1]
page_text = page.extract_text(0)
table_text = page_text[119:119 + 454]
table_text = table_text.replace('\n', ' ').replace(
    'Preferred foot', 'Preferred-foot').split(' ')
features = table_text[:6]
data = table_text[6:]
#display(feature_names)
#display(data)

##### Now it's time to get to work

In [8]:
n_features = 6
n_samples = ord('N') - ord('A') + 1

data_dict = dict()
for i, name in enumerate(features):
    data_dict[name] = [data[i + n_features * j] for j in range(n_samples)]

df = pd.DataFrame(data_dict)
df.head(3)

,Player,League,Position,Preferred-foot,Capped,Shortlisted
0,A,SerieA,CF,Left,yes,True
1,B,LaLiga,LW,Right,no,True
2,C,LaLiga,CF,Right,yes,True


In [9]:
# Let's actually delete the player column cause it is of no interest to us
# and also take target feature out of features list
df = df.iloc[:, 1:]
features = features[1:]
target_feature = features.pop()
n_features -= 2
df.head(3)

,League,Position,Preferred-foot,Capped,Shortlisted
0,SerieA,CF,Left,yes,True
1,LaLiga,LW,Right,no,True
2,LaLiga,CF,Right,yes,True


In [10]:
def compute_entropy(
    dataset: pd.DataFrame,
    target_feature: str,
    n_labels: int = 2,
    log_base: str = '2',
)->float:
    """
    Computes entropy.

    Args:
        dataset: Dataset to compute entropy for.
        target_feature: Name of the target feature.
        n_labels: Number of unqiue labels in the target feature.
        log_base: base to use for the log function. Defaults to 2.

    Returns:
        Entropy.
    """
    n_samples = len(dataset)
    counts = dataset[target_feature].value_counts().tolist()
    counts = np.pad(counts, (0, n_labels - len(counts)))
    probs = counts / n_samples + 1e-15
    log_fn = np.log2 if log_base == '2' else np.log
    entropy = -np.sum(probs * log_fn(probs))
    return entropy


def compute_information_gain(
    dataset: pd.DataFrame, 
    features: List[str],
    target_feature: str,
    n_labels: int = 2,
    log_base:str = '2',
)->List[float]:
    """
    Computes information gain.
    
    Args:
        dataset: Dataset in the form of a pandas dataframe.
        features: List of feature names excluding the target feature.
        target_feature: Name of the target feature
        n_labels: Number of unqiue labels in the target feature.
        log_base: base to use for the log function. Defaults to 2.
    
    Returns:
        A list of floats representing information gain per feature.
        Has one-to-one correspondence with the features list passed.
    """
    overall_entropy = compute_entropy(dataset, target_feature, log_base=log_base)

    ig_per_feature = []
    for feature in features:
        remainder = 0.
        for category in dataset[feature].unique():
            partition = dataset[dataset[feature] == category]
            entropy = compute_entropy(
                partition,
                target_feature,
                n_labels=n_labels,
                log_base=log_base
            )
            remainder += len(partition)/len(dataset) * entropy
        ig_per_feature.append(overall_entropy - remainder)
        
    return ig_per_feature


def compute_gini_impurity(
    dataset: pd.DataFrame,
    features: List[str],
    target_feature: str,
    n_labels: int = 2
)->List[float]:
    """
    Computes gini impurity.
    
    Args:
        dataset: Dataset in the form of a pandas dataframe.
        features: List of feature names excluding the target feature.
        target_feature: Name of the target feature
        n_labels: Number of unqiue labels in the target feature.
    
    Returns:
        A list of floats representing gini impurity per feature.
        Has one-to-one correspondence with the features list passed.
    """
    gini_impurity_per_feature = []
    for feature in features:
        weighted_impurity = 0.
        for category in dataset[feature].unique():
            partition = dataset[dataset[feature] == category]
            label_counts = partition[target_feature].value_counts().tolist()
            label_counts = np.pad(label_counts, (0, n_labels - len(label_counts)))
            probs = label_counts / len(partition)
            gini_impurity = 1.0 - np.sum(np.square(probs))
            weighted_impurity += len(partition)/len(dataset) * gini_impurity
        gini_impurity_per_feature.append(weighted_impurity)
    return gini_impurity_per_feature

##### Verifying the results (as a sanity check)
Will use natural log instead of log2 in my own implemention as well JUST for the
sake of comparison becuase sklean only computes info gain with natural log

In [11]:
# ===================== 
# SCIKIT-LEARN
# =====================
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
X = df.iloc[:, :-1]
ordinal_enc = OrdinalEncoder()
X = ordinal_enc.fit_transform(X)
y = df.iloc[:, -1]
label_enc = LabelEncoder()
y = label_enc.fit_transform(y)
info_gain = mutual_info_classif(X, y, discrete_features=True)
print("[SKLEARN] Information Gain for each feature:", info_gain)

# ======================
# MY IMPLEMENTATION
# ======================
ig_per_feature = compute_information_gain(df, features, target_feature, log_base='e')
print("[MY IMPLEMENTATION] Information Gain for each feature:", ig_per_feature)

# incase you may wonder if there's small deviation
print("\n[Me]: Dear computer, my eyes hurt comparing floats, "
      "can you please verify if all the values are close enough? Thanks! ")
print("[Computer]: They indeed are!" if np.allclose(ig_per_feature, info_gain) else "Nope!")

[SKLEARN] Information Gain for each feature: [0.10316446 0.13110121 0.17871515 0.        ]
[MY IMPLEMENTATION] Information Gain for each feature: [np.float64(0.10316445961386744), np.float64(0.1311012079924112), np.float64(0.17871515126363802), np.float64(0.0)]

[Me]: Dear computer, my eyes hurt comparing floats, can you please verify if all the values are close enough? Thanks! 
[Computer]: They indeed are!


### Root node selection procedure

#### Pseudo code
    function select_root_node(dataest, features, target_feature, n_labels, criterion):
        // criterion must be in [ig, gini]
    
        criterion_fn = information_gain if criterion is ig else compute_gini_impurity
        
        criterion_values = criterion_fn(dataset, features, target_feature, n_labels)
        
        if criterion is gini:
            feature_index = argmin(criterion_values)
        
        else:
            feature_index = argmax(criterion_values)
        
        return features[feature_index]

#### Implementation:

In [12]:
def select_root_node(
    dataset: pd.DataFrame,
    features: List[str],
    target_feature: str,
    n_labels: int = 2,
    criterion: str = 'gini'):
    assert criterion.lower() in ['ig', 'gini']
    criterion_fn = (
        compute_gini_impurity if criterion == 'gini'
        else compute_information_gain
    )
    criterion_values = criterion_fn(dataset, features, target_feature, n_labels)
    if criterion == 'gini':
        feature_index = np.argmin(criterion_values)
    else:
        feature_index = np.argmax(criterion_values)
    return features[feature_index]
        

In [13]:
features = ['League', 'Position', 'Preferred-foot', 'Capped']
target_feature = 'Shortlisted'

for criterion in ['gini', 'ig']:
    print(f"Root node using {criterion} as criterion: ", end="")
    print(select_root_node(df, features, target_feature, criterion=criterion))

Root node using gini as criterion: Preferred-foot
Root node using ig as criterion: Preferred-foot


# Problem 2

In [14]:
# For this problem, we'll import the wdbc dataset

wdbc_train_df = pd.read_csv("./wdbc/wdbc_train.csv")
wbdc_dev_df = pd.read_csv("./wdbc/wdbc_dev.csv")
wdbc_test_df = pd.read_csv("./wdbc/wdbc_test.csv")

wdbc_train_df.shape, wbdc_dev_df.shape, wdbc_test_df.shape

((341, 31), (114, 31), (114, 31))

skipping data processing as it is not a concern for the sake of this assignment

## ID3 implemention 

In [15]:
from enum import Enum

In [16]:
NodeType = Enum('NodeType', ['ROOT', 'INTERMEDIATE', 'LEAF'])

In [17]:
class Node:
    """
    Node class specifically developed for the implemention of id3 algorithm 
    in this notebook.

    Args:
        name: Name or label of the node.
        majority_label: Majority label in the population where the node exists.
        parent: Parent of the node. For root it is None.
        why: Why was the node created. Serves as the edge label from parent to
            the child node.
        node_type: What type of node it is? One of [ROOT, INTERMEDIATE, LEAF]
        depth: Depth at which the node occurs.
    """
    def __init__(
        self,
        name: str, 
        majority_label:str,
        parent=None,
        why: str = "404",
        node_type=NodeType.INTERMEDIATE,
        depth: int = 0,
    ):
        self.name = name
        self.parent = parent
        self.depth = depth
        self.why = why
        self.children = []
        self.node_type = node_type
        self.majority_label = majority_label
        self.mapping = dict()
        if parent:
           self.parent.add_child(self) 
    
    def add_child(self, child_node):
        """
        Adds a child to the current node and updates self.mapping for inference.

        Args:
            child_node: Child node to be added to as a child.
        """
        child_node.depth = self.depth + 1
        self.mapping[child_node.why] = child_node
        self.children.append(child_node)

    def predict(self, x: dict):
        """
        Recursively goes through all the nodes depending on the given instance 
        and returns the outcome/label.

        Args:
            x: A data instance. Must be dict or dict-like.

        Returns:
            Resulting label based on the instance.
        """
        if self.node_type == NodeType.LEAF:
            return self.name
        value = x[self.name]
        # Return majority label when an unkown value is encountered
        if value not in self.mapping:
            return self.majority_label
        next_node = self.mapping[value]
        return next_node.predict(x)

#### id3 implementation allowing both information gain and gini 

In [18]:
def id3(
    dataset: pd.DataFrame,
    features: List[str],
    target_feature: str,
    parent_node: Node = None,
    why:str = '404',
    criterion:str = "ig",
)->Node:
    """
    Khawaja's implementation of ID3 algorithm based on the psuedocode in the
    book/slides allowing both information gain and gini as criterions.
    This implementation utilizes pandas in addition to numpy to make life 
    easier.
    
    Args:
        dataset: Must be pandas dataframe.
        features: List of names of features in the dataset.
        target_feature: Name of target feature.
        parent_node: Node that called this function. For the first call it would
            be None.
        why: why was the node created, or what led to the creation of node.
            In simpler words, it's the edge label that connects parent node 
            with the child node
        criterion: Criteria to use for node splitting. One of ['ig', 'gini']

    Returns:
        The root node of the tree.
    """
    assert criterion in ['ig', 'gini']
    features = copy(features) # creates copy to avoid changes inplace
    label_counts = dataset[target_feature].value_counts().tolist()
    majority_label = dataset[target_feature].unique().tolist()[np.argmax(label_counts)]
    
    if criterion == 'ig':
        criterion_fn = compute_information_gain
    elif criterion == 'gini':
        criterion_fn = compute_gini_impurity
    else:
        raise ValueError("Invalid value for criterion.")
    
    # Base case 1:
    # If all the instances in dataset have the same target label then
    # return a decision tree witha leaf node 
    if len(label_counts) == 1:
        new_node = Node(
            name=majority_label,
            majority_label=majority_label,
            parent=parent_node,
            why=why,
            node_type=NodeType.LEAF)
        return parent_node if parent_node else new_node

    # Base case 2:
    # If the feature set being considered is empty, return a decision tree with
    # leaf node with the majority label in dataset
    elif len(features) == 0:
        new_node = Node(
            name=majority_label,
            majority_label=majority_label,
            parent=parent_node,
            why=why,
            node_type=NodeType.LEAF)
        return parent_node if parent_node else new_node
    
    # Base case 3:
    # If the dataset is empty then return a decision tree with a leaf node with
    # the label of the majority label in the data partition of the immediate
    # parent
    elif len(dataset) == 0:
        new_node = Node(
            name=parent_node.majority_label,
            majority_label=None,
            parent=parent_node,
            why=why,
            node_type=NodeType.LEAF
        )
        return parent_node if parent_node else new_node

    # Else (now the fun part starts)
    criteria_vals = criterion_fn(
        dataset,
        features,
        target_feature)
    #print(f"{criteria_vals=}")
    if criterion == "gini":
        idx = np.argmin(criteria_vals)
    else:
        idx = np.argmax(criteria_vals)
    #print(f"{idx}")
    fbest = features[idx]
    new_node = Node(
        name=fbest,
        majority_label=majority_label,
        parent=parent_node,
        why=why,
        node_type=NodeType.INTERMEDIATE)
    #print("fbest: ", fbest)
    features.remove(fbest)
    for category in dataset[fbest].unique():
        partition = dataset[dataset[fbest] == category]
        id3(partition, features, target_feature, parent_node=new_node,
            why=category, criterion=criterion)
    return parent_node if parent_node else new_node

In [19]:
target_feature = 'Diagnosis'
features = list(set(wdbc_train_df.columns) - set([target_feature]))
wdbc_tree = id3(wdbc_train_df, features, target_feature)

#### Tree graph plotting using graphviz
I'd like to mention that I did take a shot at developing my own function for tree visualization which worked for very small trees but failed at large tree. The code is available in the accompanying `idai610-ps1-brainstorm.ipynb` notebook.

In [20]:
from graphviz import Digraph

def trace(root):
    nodes, edges, edge_labels = set(), set(), list()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v.children:
                edges.add((v, child))
                edge_labels.append(child.why)
                build(child)
    build(root)
    return nodes, edges, edge_labels

def draw_graph(root):
    nodes, edges, edge_labels = trace(root)
    graph = Digraph(format='png', graph_attr={'rankdir': 'TB'})

    for n in nodes:
        graph.node(name=str(id(n)), label=n.name, shape='record')
    for (n1, n2), label in zip(edges, edge_labels):
        graph.edge(str(id(n1)), str(id(n2)), label=str(label))
    return graph

In [21]:
#tree = id3(df, features, target_feature, None, criterion="ig")
#draw_graph(tree)

#### Alright buckle up, let's implement pruining!

In [22]:
from scipy.stats import chi2

def prune_tree(
    root: Node,
    dataset: pd.DataFrame,
    target_feature: str,
    significance_level: float = 0.05,
    max_pruning_depth: int = 1,
):
    """
    Assumes binary target feature.

    PLEASE NOTE: This function prunes the given tree in-place!

    Args:
        root: Root node of the given decision tree.
        dataset: Dataset used in building of the given decision tree. It must be
            the exact same dataset.
        target_feature: Name of the target feature.
        significance_level: Significance level for the statistical test.
        max_pruning_depth: The maximum depth to which the tree can be pruned.
            Remember the root is at depth 0, then 1, 2, 3... and so on.
    """
    if len(root.children) == 0:
        return root
    visited_nodes_idx = []
    penultimate_nodes = []
    def dig(root):
        for child in root.children:
            if child.node_type == NodeType.LEAF:
                if child.parent not in penultimate_nodes:
                    penultimate_nodes.append(child.parent)
            else:
                dig(child)
    dig(root)
    
    # would obviously break if the target feature isn't binary
    # Note: I'm following nomenclature used in the R&N book
    p, n = dataset[target_feature].value_counts().to_list()
    labels = dataset[target_feature].unique()
    
    # Let's begin pruning
    while True:
        node = penultimate_nodes.pop(0)
        if node not in visited_nodes_idx:
            visited_nodes_idx.append(str(id(node)))
        else:
            if len(penultimate_nodes) == 0:
                break
            continue
        if node.parent == None:
            if len(penultimate_nodes) == 0:
                break
            continue
        delta = 0.
        degrees_of_freedom = dataset[node.name].nunique() - 1
        for category in dataset[node.name].unique():
            partition = dataset[dataset[node.name]==category]
            if partition[target_feature].nunique() == 1:
                if partition[target_feature].unique()[0] == labels[0]:
                    pk = partition[target_feature].value_counts().tolist()[0]
                    nk = 0
                else:
                    nk = partition[target_feature].value_counts().tolist()[0]
                    pk = 0
            else:
                pk, nk = partition[target_feature].value_counts().tolist()
            pk_hat = (len(partition) / len(dataset)) * p
            nk_hat = (len(partition) / len(dataset)) * n
            delta += (pk - pk_hat)**2 / pk_hat + (nk - nk_hat)**2 / nk_hat
        threshold = chi2.ppf(1.0 - significance_level, degrees_of_freedom)
        if delta < threshold:
            # prune it away
            if node in node.parent.children:
                node.parent.children.remove(node)
            new_node = Node(
                name=node.majority_label,
                majority_label=node.majority_label,
                parent=node.parent,
                why=node.why,
                node_type=NodeType.LEAF,
            )
            penultimate_nodes.append(node.parent)
            for child_node in node.children:
                del child_node
            del node
        
        if len(penultimate_nodes) == 0:
            break    

In [23]:
prune_tree(wdbc_tree, wdbc_train_df, target_feature)

##### Please note that I'm not visualizing the tree before and after pruning in this notebook to keep it concise as visualization is not a requirement of the given problem. But I do visualize trees in my brainstorm notebook also included in the zip file.

# Problem 3 

## Use your decision tree to develop a model for WDBC

### Train

In [24]:
import pandas as pd

wdbc_train_df = pd.read_csv("./wdbc/wdbc_train.csv")

In [25]:
# wdbc_train_df.describe()

In [26]:
target_feature = 'Diagnosis'
features = list(set(wdbc_train_df.columns) - set([target_feature]))

In [27]:
tree = id3(wdbc_train_df, features, target_feature)

### Function for prediction on batch using our tree

In [28]:
def tree_batch_predict(decision_tree, x_batch: pd.DataFrame)->np.array:
    return np.asarray(list(map(decision_tree.predict, x_batch.to_dict('records'))))

### Tuning

Tuning focuses on selecting the best node-splitting criterion (Gini or IG).

My implemention also incorporates the selection of chi-squared pruning.

In [29]:
wdbc_dev_df = pd.read_csv("./wdbc/wdbc_dev.csv")
wdbc_dev_df.shape

(114, 31)

In [30]:
from copy import deepcopy, copy

def binary_accuracy(
    y_true: np.array,
    y_pred: np.array
):
    """
    Computes the binary accuracy using the categorical values (not probabilites)

    Args:
        y_true: numpy array of true values.
        y_pred: numpy array of predicted values.

    Returns:
        Binary ccuracy (a float value).
    """
    return np.mean(y_true == y_pred)

def tree_tuner(
    tree_builder: callable,
    train_dataset: pd.DataFrame,
    dev_dataset: pd.DataFrame,
    features: List[str],
    target_feature: str,
    objective_fn: callable = binary_accuracy,
    objective_type: str = 'max', # minimize or maximize
):
    '''
    tree tuner aims the find the best tree with the best config from a given
    space of hyperparameters.

    Args:
        tree_builder: A function that builds the tree. For instance, id3.
        train_dataset: training dataset. Must be pandas dataframe.
        dev_dataset: Dataset for tuning hyperparameters.
        features: List of names of features in the dataset.
        target_feature: Name of the target feature.
        objective_fn: Objective function that you're trying to optimize the tree
            for.
        objective_type: Whether to maximize the objective value or minimize it.
            Must be in ['min', 'max'].

    Returns:
        A tuple of best performing tree (i.e the root node), best config as a
        dict and best score value.
    '''
    assert objective_type in ['min', 'max']
    
    best_tree = None
    curr_config = dict()
    best_config = dict()
    best_score = -np.inf if objective_type == 'max' else np.inf
    trial = 0
    # Obviously not the best way to do 'grid search' but hey it looks more
    # intutive when there' just two hyperparameters with only like 2 values each
    for criterion in ['gini', 'ig']:
        for prune in [False, True]:
            curr_config['criterion'] = criterion
            curr_config['prune'] = prune
            # train
            tree = tree_builder(
                train_dataset,
                copy(features),
                target_feature,
                criterion=criterion
            )
            if prune:
                # Remember prune_tree prunes the tree in place
                prune_tree(
                    tree,
                    train_dataset,
                    target_feature,
                    max_pruning_depth=1
                )
            # evaluate on dev
            y_pred = tree_batch_predict(tree, dev_dataset)
            score = objective_fn(
                dev_dataset[target_feature].values,
                y_pred
            )
            if objective_type == 'max' and score > best_score:
                best_tree = deepcopy(tree)
                best_config = curr_config
                best_score = score
            print(f"Trial: {trial} | Score: {score} | Best Score: {best_score}", end="")
            print(f" | Current Config: {curr_config} | Best Config: {best_config}")
            trial += 1
    return best_tree, best_config, best_score

In [31]:
target_feature = 'Diagnosis'
features = list(set(wdbc_dev_df.columns) - set([target_feature]))

best_tree, best_config, best_score = tree_tuner(
    id3,
    wdbc_train_df,
    wdbc_dev_df,
    features,
    target_feature
)

Trial: 0 | Score: 0.9649122807017544 | Best Score: 0.9649122807017544 | Current Config: {'criterion': 'gini', 'prune': False} | Best Config: {'criterion': 'gini', 'prune': False}
Trial: 1 | Score: 0.9649122807017544 | Best Score: 0.9649122807017544 | Current Config: {'criterion': 'gini', 'prune': True} | Best Config: {'criterion': 'gini', 'prune': True}
Trial: 2 | Score: 0.9649122807017544 | Best Score: 0.9649122807017544 | Current Config: {'criterion': 'ig', 'prune': False} | Best Config: {'criterion': 'ig', 'prune': False}
Trial: 3 | Score: 0.9649122807017544 | Best Score: 0.9649122807017544 | Current Config: {'criterion': 'ig', 'prune': True} | Best Config: {'criterion': 'ig', 'prune': True}


Developer Note: It is just weird that the results here are exactly the same.

Please refer to the brainstorm notebook where I demonstrate both by visualizing and computing metrics that the pruning works and also gini and ig do result in different results sometimes (though for small they tend to produce exactly the same results.

### Test

In [32]:
test_df = pd.read_csv("./wdbc/wdbc_test.csv")
test_df.shape

(114, 31)

In [33]:
test_accuracy = np.mean(test_df[target_feature] == tree_batch_predict(best_tree, test_df))
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.8947368421052632


Not bad, eh?

# [Optional] Discretize features

I'm using a very simple approach of binning the continuous features into categorical, specifically using the equal-width approach which works by discretizing continuous data by dividing the range of possible values into a specified number of intervals (bins) of equal size.

In [34]:
def categorize_features_equal_width(dataset, n_bins=5):
    """
    Bins continuous values into categorical.
    
    Args:
        X: pandas dataset with continuous features
        n_bins: Number of bins to use for categorization
    
    Returns:
        A 2d numpy array of categroized features.
    """
    dataset_arr = dataset.values
    n_samples, n_features = X.shape
    dataset_categorical = np.zeros_like(dataset_arr, dtype=int)
    
    for feature in range(n_features):
        feature_values = dataset_categorical[:, feature]
        bins = np.linspace(feature_values.min(), feature_values.max(), n_bins + 1)
        
        dataset_categorical[:, feature] = np.digitize(feature_values, bins[1:-1])
    
    return dataset_categorical